In [17]:
import os
import iucrime.parse

In [18]:
dates = ['1-1-13', '1-13-16', '2-8-14']
path = '../../data'

Let's just pick one data file and take a look

In [12]:
fname = os.path.join(path, 'text', dates[0] + '.txt')
fname

'../../data/text/1-1-13.txt'

In [14]:
print(open(fname).read())

Indiana University, Bloomington
Police Department
Student Right To Know CAD Daily Log

From Jan 01, 2013 to Jan 01, 2013.
Date Reported: 01/01/13 - TUE at 23:49
Location : INDIANA MEMORIAL UNION
Date and Time Occurred From - Occurred To
01/01/13 - TUE at 23:45 - 01/01/13 - TUE at 23:52
Incident : ALL OTHER OFFENSES - TRESPASS & MALICIOUS TRESPASS
Disposition: CLOSED BY ARREST

Event #: 13-01-01-000047
Report #: 130002

1 Incidents Listed.

Print Date and Time

1/2/2013

6:02:05PM

at Page No.

1


